## Start

In [2]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)



___
### Tags: #Source #Bitrix

### Links: 
___

In [9]:
# creating a table from s3

query_text = """--sql 
   CREATE TABLE db1.category_id_dir_bitrix
(
    `category_id` UInt32,
    `category_name` String,
    `stage_id` String,
    `name` String
)
ENGINE = S3('https://storage.yandexcloud.net/aggregated-data/bitrix_category_id/*.csv','CSVWithNames')
SETTINGS format_csv_delimiter = ';'
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

In [15]:
query_text = """--sql
    CREATE TABLE db1.category_id_dir_bitrix_ch
(
    `category_id` UInt32,
    `category_name` String,
    `stage_id` String,
    `name` String
)
    ENGINE = MergeTree()
    ORDER BY category_id
    
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

In [10]:
query_text = """--sql
    CREATE MATERIALIZED VIEW db1.category_id_dir_bitrix_mv
    REFRESH EVERY 1 DAY OFFSET 3 HOUR RANDOMIZE FOR 1 HOUR TO db1.category_id_dir_bitrix_ch AS
    SELECT
        category_id,
        category_name,
        if(category_id != 0, splitByChar(':', stage_id)[2],stage_id) AS stage_id,
        name
    FROM db1.category_id_dir_bitrix
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

___
## Tools
___
### query


In [19]:
query_text = """--sql
    SELECT
        *
    FROM db1.category_id_dir_bitrix_ch
    limit 100
    """

ch.query_run(query_text)


category_id,category_name,stage_id,name
i64,str,str,str
21,"""Перемещения ЦС""","""C21:NEW""","""Новая"""
21,"""Перемещения ЦС""","""C21:PREPARATION""","""Выставлен счёт"""
21,"""Перемещения ЦС""","""C21:PREPAYMENT_INVOICE""","""Cчёт оплачен"""
21,"""Перемещения ЦС""","""C21:WON""","""Сделка успешна"""
21,"""Перемещения ЦС""","""C21:LOSE""","""Сделка провалена"""
…,…,…,…
31,"""Продажи РТК""","""C31:PREPAYMENT_INVOIC""","""Выставлен счет клиенту"""
31,"""Продажи РТК""","""C31:EXECUTING""","""На оплате у клиента"""
31,"""Продажи РТК""","""C31:WON""","""Сделка успешна"""


### drop mv

In [7]:
query_text = """--sql
    DROP TABLE db1.category_id_dir_bitrix_mv
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### drop ch

In [ ]:
query_text = """--sql
    DROP TABLE db1.cameras_dir_asgard_ch
    """

ch.query_run(query_text)

### refresh mv

In [ ]:
query_text = """
SYSTEM REFRESH VIEW db1.cameras_dir_asgard_mv
"""

ch.query_run(query_text)